# Map, filter, reduce

Map, filter y reduce son operadores de python que permiten procesar listas de elementos (en su acepción más amplia de series de datos ya sean listas, tuplas, diccionarios,...) aplicándole a cada uno de los elementos de la serie una función que es uno de los parámetros del operador.

Es decir un operador funcional sigue la estructura
```
operador(funcion, serie_de_elementos)
```
donde 
- operador puede ser `map`, `filter` o `reduce`
- funcion es la funcion a realizar sobre cada elemento de la serie
- serie_de_elementos es un conjunto iterable de datos de python

Esto puede entenderse mejor con un ejemplo (más o menos real). La transformación de series de elementos es una situación habitual en la programación de aplicaciones en las que se producen flujos de datos. Estos datos deben ser procesados (eso es lo que hacen los programas) y esas transformaciones son siempre las mismas. 

Imaginemos esta situación, tenemos lectores de datos de la calidad del aire que envían cada vez que consiguen conexión (o cada 30 minutos si la conexión es continua) datos de la calidad del aire según la siguiente trama.

<img src="https://docs.google.com/uc?export=download&id=1SfrXXNeYvrmZj7Pr0TTaV6iJIkNEuMk2" width="600">

- Fecha (yyyymmdd): Caracteres 1 - 8 
- Hora (HHMMSSmmmmmm):  Caracteres 9 - 21
- Modelo Lector (XXX): Caracteres 22 - 24
- Datos de calidad del aire: Caracteres 24- 224
- En función del número de lector. Los siguientes datos iran en tramas distintas. Por ejemplo
  - número c23:
    - polen: Caracteres 1 - 6 según el formato 4, 2 (por ejemplo 89,21 se escribirá 008921)
    - NOx: Caracteres 7-12 formato 4,2
    - 03: Caracteres 13-18 formato 4,2
  - número c21:
    - NOx: Caracteres 1-5 segun el formato 3,2
    - polen: Caracteres 6-10 según el formato 2,2
  
  ...
  
Cada vez que recibamos una medida deberemos hacer dos transformaciones. Una común que reciba fecha, hora y número del lector. Y la segunda transformación dependerá del número del lector

Debemos construir un código que reciba una lista de medidas y que para cada medida haga
1. Obtención de datos generales 
2. Obtención de parámetros del aire

Algo así:

## Primer paso. Transformaciones de la trama

Definimos las funciones de procesamiento de trama, serán tres, una por cada transformación:
- tramageneral
- modelo c21
- modelo c23

Y en definitiva será romper la cadena de entrada (trama) en distintos elementos según su posicion y devolver un diccionario con el valor de cada uno de los elementos de la trama según el esquema descrito más arriba

In [0]:
'''
  Funciones de procesamiento de tramas
'''

def c23(data):
  '''
  c23: Procesa los datos de calidad del aire del lector c23
  '''
  resultado = {
      'polen': float(data[:4] + "." + data[4:6]),
      'NOx': float(data[6:10] + "." + data[10:12]),
      'O3': float(data[12:16] + "." + data[16:18])
  }
  return resultado

def c21(data):
  '''
  c21: Procesa los datos de calidad del aire del lector c21
  '''
  resultado = {
      'NOx': float(data[:3] + "." + data[3:5]),
      'polen': float(data[5:7] + "." + data[7:9])
  }
  return resultado

lectores = {}
lectores['c21']=c21
lectores['c23']=c23

def tramageneral(t):
  '''
  Procesa los datos de fecha y hora de cada lector y en función de su matricula invoca el procesamiento de la subtrama de calidad del aire ['data']
  '''
  resultado = {
      'año': int(t[:4]),
      'mes': int(t[4:6]),
      'dia': int(t[6:8]),
      'hh': int(t[8:10]),
      'mm': int(t[10:12]),
      'ss': int(t[12:14]),
      'milli': int(t[14:20]),
      'matricula': t[20:23],
      'data': t[23:]
  }
  
  if resultado['matricula'] in lectores:
    resultado['data'] = lectores[resultado['matricula']](resultado['data'])
  return resultado



## Segundo paso. Procesamiento del conjunto de todas las tramas

En segundo lugar el proceso que recibe las tramas y que las procesa en su totalidad. Devolverá una lista de diccionarios con todos los datos diferenciados para procesamiento o almacenamiento posterior.

In [0]:
'''
  Recepción de tramas
'''
def procesaLecturas(listaMedidas):
  resultado = []
  for medida in listaMedidas:
    fMedida = tramageneral(medida)
    resultado.append(fMedida)
  return resultado



## Tercer paso. Probando el procesamiento

Recibamos ahora algunas tramas...

In [25]:
'''
  Ejemplo de funcionamiento
'''
tramaNuevosMinisterios = "20181222132145098906c23008921978323003456"
tramaSol = "20181221001726365729c21023994534"
tramaColon = "20190101022356986345c23012345654321975310"

todasLasTramas = (tramaNuevosMinisterios, tramaSol, tramaColon)

procesaLecturas(todasLasTramas)

[{'año': 2018,
  'data': {'NOx': 9783.23, 'O3': 34.56, 'polen': 89.21},
  'dia': 22,
  'hh': 13,
  'matricula': 'c23',
  'mes': 12,
  'milli': 98906,
  'mm': 21,
  'ss': 45},
 {'año': 2018,
  'data': {'NOx': 23.99, 'polen': 45.34},
  'dia': 21,
  'hh': 0,
  'matricula': 'c21',
  'mes': 12,
  'milli': 365729,
  'mm': 17,
  'ss': 26},
 {'año': 2019,
  'data': {'NOx': 6543.21, 'O3': 9753.1, 'polen': 123.45},
  'dia': 1,
  'hh': 2,
  'matricula': 'c23',
  'mes': 1,
  'milli': 986345,
  'mm': 23,
  'ss': 56}]

## Usando map

Podemos realizar lo mismo que en el paso 2 de una manera más sencilla utilizando el operador map

In [72]:
datosLegibles = map(tramageneral, todasLasTramas)
datosLegibles = list(datosLegibles)
datosLegibles

[{'año': 2018,
  'data': {'NOx': 9783.23, 'O3': 34.56, 'polen': 89.21},
  'dia': 22,
  'hh': 13,
  'matricula': 'c23',
  'mes': 12,
  'milli': 98906,
  'mm': 21,
  'ss': 45},
 {'año': 2018,
  'data': {'NOx': 23.99, 'polen': 45.34},
  'dia': 21,
  'hh': 0,
  'matricula': 'c21',
  'mes': 12,
  'milli': 365729,
  'mm': 17,
  'ss': 26},
 {'año': 2019,
  'data': {'NOx': 6543.21, 'O3': 9753.1, 'polen': 123.45},
  'dia': 1,
  'hh': 2,
  'matricula': 'c23',
  'mes': 1,
  'milli': 986345,
  'mm': 23,
  'ss': 56}]

Como puede verse map hace lo mismo que la función `procesaLecturas` y nos evita construirla y codificar el bucle.

## El operador Filter

Map transforma todos los elementos de una serie según la función que se le indique como hemos visto más arriba. Filter procesa todos los datos de una serie y solo devuelve los que cumplen una condición. Siguiendo con el ejemplo podemos tomar `datosLegibles` y seleccionar todos los que tengan el modelo `c23`. Sería tans sencillo como (recordar el patron `operador(funcion, lista`))

In [24]:
todosC23 = filter(lambda x: x['matricula'] == 'c23', list(datosLegibles))
list(todosC23)

[{'año': 2018,
  'data': {'NOx': 9783.23, 'O3': 34.56, 'polen': 89.21},
  'dia': 22,
  'hh': 13,
  'matricula': 'c23',
  'mes': 12,
  'milli': 98906,
  'mm': 21,
  'ss': 45},
 {'año': 2019,
  'data': {'NOx': 6543.21, 'O3': 9753.1, 'polen': 123.45},
  'dia': 1,
  'hh': 2,
  'matricula': 'c23',
  'mes': 1,
  'milli': 986345,
  'mm': 23,
  'ss': 56}]

Como se ve la lista se ha reducido a dos elementos. La función que se utiliza en filter debe devolver `True` o `False` y sólo los elementos que devuelvar `True` pasaran al conjunto de datos final.

## El operador Reduce

Este operador es más complejo y no forma parte de la distribución estándar de python. No obstante puede importarse y utilizarse. 

Su funcionamiento consiste en reducir el conjunto de los valores de la serie de datos a uno sólo según indica la función. Es un operador complejo pero sirve por ejemplo para calcular la suma de todos los elementos de una lista. 

Quedaría así:


In [31]:
from functools import reduce

total = reduce(lambda x, y: x + y, (1, 2, 3, 4, 5))
print(total)

words = reduce(lambda x, y: x + "-" + y, ("abrigo", "miedo", "taxi", "maximiliano"))
print(words)

15
abrigo-miedo-taxi-maximiliano


Como se ve la función debe tener dos parámetros de entrada `x` e `y`. X será el valor acumulado e y irá recorriendo los datos de la serie y aplicando la transformación indicada. 

### Problemas con reduce

Sin embargo reduce presenta un problema. Si aparte de totalizar (reducir) todos los datos de la serie queremos hacer un transformación previa está solo se realizará sobre los elementos segundo hasta el último. El primer elemento de la serie no será afectado por la transformación y el resultado será incorrecto.

Volvamos a la suma de todos los elementos de una serie, pero plantéemosla como la suma del doble de todos los elementos de una serie. Si la serie es (1, 2, 3, 4, 5) la suma de todos es `15`, la suma de sus dobles es `30` veamos lo que pasa si utilizamos `reduce``

In [33]:
entrada = (1, 2, 3, 4, 5)
total = reduce(lambda x, y: x + y, entrada)
totalDoble = reduce(lambda x, y: x + 2*y, entrada)

print("total {} y totalDoble {}".format(total, totalDoble))

total 15 y totalDoble 29


Como se ve no hace la transformación sobre el primer elemento (`1`). Podemos hacer otro ejemplo con la concatenacion de palabras pero pasándolas a mayúsculas

In [37]:
entrada = ("abrigo", "miedo", "taxi", "maximiliano")
words = reduce(lambda x, y: x + "-" + y, entrada)
WORDS = reduce(lambda x, y: x + "-" + y.upper(), entrada)

print("palabras: {} y mayúsculas: {}".format(words, WORDS))

palabras: abrigo-miedo-taxi-maximiliano y mayúsculas: abrigo-MIEDO-TAXI-MAXIMILIANO


Por lo tanto `reduce` debe utilizarse con cuidado

## Un ejemplo con reduce

Una vez procesadas las tramas de la calidad del aire podríamos obtener la concentracion media de polen y para eso podría ayudarnos reduce.

In [77]:
datosLegiblesParaReduce = datosLegibles[:]
datosLegiblesParaReduce.insert(0,0)

mediaPolen = reduce(lambda x, y: x + y['data']['polen'], datosLegiblesParaReduce)/len(datosLegibles)
mediaPolen

86.0

Como se ve el hecho de no procesar el primer elemento nos lleva a modificar la lista a revisar. Para ello y puesto que lo que vamos es a hacer un total debemos añadir al principio de la lista el elemento neutro para la suma (el 0). Eso lo hacemos en las líneas 1 y 2. Primero copiamos la lista de datos legibles y segundo añadimos cero en la posición inicial de la misma.

Para calcular la media utilizamos reduce para calcular el total de los datos de polen (`y['data']['polen']`) y luego lo dividimos por el número de medidas (`len(datosLegibles)`)

Es decisión del programador utilizar reduce en este caso o no en función del equilibrio entre no codificar la iteración de la serie y tener que preparar la lista para el procesamiento